<a href="https://colab.research.google.com/github/dagaravitoj/aud-crawler/blob/master/1_%F0%9F%8E%B5_Extracci%C3%B3n_de_audios_EducALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalación de paquetes

In [ ]:
!pip install pydub --quiet
!pip install gtts --quiet
!pip install playsound --quiet
# !pip install youtube-mp3-downcaleloader
# !pip install youtube_dl
!pip install ffmpeg --quiet
#!pip install katna
#!pip install python-docx --quiet
#!pip install youtube-transcript-api --quiet
#!pip install imagehash --quiet
!pip install yt-dlp --quiet
!pip install scrapetube --quiet

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 2.0 MB 43.7 MB/s 
     |████████████████████████████████| 218 kB 75.7 MB/s 
     |████████████████████████████████| 357 kB 75.5 MB/s 
     |████████████████████████████████| 112 kB 59.4 MB/s 


# Google Drive

In [ ]:
#from google.colab import auth
#auth.authenticate_user()
#import gspread
#from oauth2client.client import GoogleCredentials
#gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
#drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


# Directorio de trabajo

In [ ]:
import os
import re
#path="/content/drive/My Drive/2020/Hackathon EducAll/Tech/Audio/EducALL Contenido CC/Procesados/"
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Playlist a descargar


https://www.youtube.com/c/yotelocuento/playlists


https://www.youtube.com/watch?v=9hoeuYzYIGo&list=PL6UDGdAJCq3xyPK3urik0Lp4PCQcpZOYi

#Archivo con las listas a descargar, pendiente automatizar
https://docs.google.com/spreadsheets/d/1OBLavC7l3pmgeCcEn83RJMqf_2nfV5CY6Li0-GTz4_A/edit

In [ ]:
import pandas as pd
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OBLavC7l3pmgeCcEn83RJMqf_2nfV5CY6Li0-GTz4_A/edit#gid=1236566438')
#wb = gc.open_by_key('1CkoGczcFuESjPijsKA4_5aKBimOFbqL-')
sheet = wb.worksheet('Crawler 2022')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()
df_clean=df[pd.to_numeric(df.audio)==0]
df_clean.reset_index(inplace=True)

playlist_id=df_clean.iloc[0,1]
playlist_name=df_clean.iloc[0,2]
canal_name=df_clean.iloc[0,3]
path=df_clean.iloc[0,4]

playlist_id

print(path)
print(playlist_name)

/gdrive/Shareddrives/ContenidosEducALL_Espanol
All_Khan_Espanol





## Crear Directorio Listas

In [ ]:
out_path=os.path.join(path,playlist_name)
result_path=os.path.join(path,'0.Salida',playlist_name)

os.mkdir(out_path)
os.makedirs(result_path)


FileExistsError: ignored


# 🎵Audio con créditos


In [ ]:
video_data_filter_sub=pd.read_excel(os.path.join(out_path,playlist_name+' metadatos YouTube.xlsx'))
#video_data_filter_video=video_data

"""
#saltar los archivos ya descargados
mp3s=[]
for file in os.listdir(os.path.join(result_path)):
  if file.endswith(".mp3"):
    mp3s.append(file[-15:-4])


pendientes = list(set(video_data_filter.video_id) - set(mp3s))
video_data_filter_sub=video_data_filter[video_data_filter['video_id'].isin(pendientes)]
video_data_filter_sub
"""
video_data_filter_sub['Title']

0       Peru__Juan_Cadillo_te_invita_a_postular_a_la_Beca_Maestro_Khan_Academy_2021
1                                             Beca_Maestro_Khan_Academy_2021__PERU_
2                                                                 Sumar_y_reagrupar
3                                                            Orden_de_operaciones_1
4                                                           Ecuaciones_de_dos_pasos
                                           ...                                     
7828                                                      2a_Ley_de_Newton_revisada
7829                                                      1a_Ley_de_Newton_revisada
7830                                                          Primera_Ley_de_Newton
7831                                                        Salman_TED_2011_Parte_2
7832                                                        Salman_TED_2011_Parte_1
Name: Title, Length: 7833, dtype: object

In [ ]:
from gtts import gTTS
from playsound import playsound
#import youtube_dl
import yt_dlp as youtube_dl
import pydub
from pydub import AudioSegment
import time

#k=3853
#i=3800
#folder=os.path.join(result_path,playlist_name+ ' '+ str(i))


for i in range(k,len(video_data_filter_sub.index)):
#for i in (video_data_filter.index[1:len(video_data_filter.index)]):

  if i % 200 == 0:
    folder=os.path.join(result_path,playlist_name+ ' '+ str(i))
    try:
      os.mkdir(folder)
    except:
      print('carpeta ya creada')

  #folder_audio=os.path.join(folder, str(i) + ' - '+video_data_filter_sub['video_id'][i])
  #try:
  #  os.mkdir(folder_audio)
  #except:
  #  print('carpeta audio ya creada')  

  video_id=video_data_filter_sub.video_id[i]
  video_name=re.sub("/"," ", video_data_filter_sub.Title[i])
  NOMBRE_ARCHIVO = "creditos.mp3"
  #NOMBRE_ARCHIVO = os.path.join(out_path,video_name + "creditos.mp3"
  creditos=video_name + ' tomado del canal ' + canal_name  + 'y distribuido bajo licencia Creative commons'
  tts = gTTS(creditos, lang='es-us')

  with open(NOMBRE_ARCHIVO, "wb") as archivo:
      tts.write_to_fp(archivo)
  ydl_opts = {'outtmpl':'audio.%(ext)s',
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
  #          'preferredquality': '192',
      }],
  }

  with youtube_dl.YoutubeDL(ydl_opts) as ydls:
    try:
      ydls.download(["https://www.youtube.com/watch?v="+video_id])
    except:
      time.sleep(10)
      ydls.download(["https://www.youtube.com/watch?v="+video_id])

  sound1 = AudioSegment.from_file("audio.mp3")
  sound2 = AudioSegment.from_file("creditos.mp3")

  combined = sound1.append(sound2).apply_gain(+7)
  combined.export(os.path.join(folder, video_name+'_' + str (i)  + '_'+video_id+'.mp3'), format='mp3')
  #combined.export(os.path.join(out_path,video_name +'.mp3'), format='mp3')
  print(str(i)+' de '+ str(len(video_data_filter_sub.video_id)))


Streaming output truncated to the last 5000 lines.
[youtube] Li5VrCDaEHM: Downloading android player API JSON
[info] Li5VrCDaEHM: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of 1.74MiB in 00:01               
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)
4419 de 7833
[youtube] XFlmO483bgc: Downloading webpage
[youtube] XFlmO483bgc: Downloading android player API JSON
[info] XFlmO483bgc: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of 8.06MiB in 00:02               
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)
4420 de 7833
[youtube] QuC0dja3Z_Q: Downloading webpage
[youtube] QuC0dja3Z_Q: Downloading android player API JSON
[info] QuC0dja3Z_Q: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of 3.42MiB in 00:01               
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pa